In [1]:
import os,sys

from nipype.interfaces import utility as niu
from nipype.pipeline import engine as pe
from niworkflows.engine.workflows import LiterateWorkflow as Workflow

# Nodes
sys.path.append('/opt/pynordic')
from pynordic.interfaces.nordic import Nordic
from nipype.algorithms.misc import Gzip
from nipype.algorithms.confounds import TSNR
from niworkflows.interfaces.nibabel import GenerateSamplingReference

In [2]:
def _get_sbref(bold):

    return bold.replace('_bold.nii.gz','_sbref.nii.gz')

def _get_split_volume(out_files,vol_id):

    return out_files[vol_id]

def init_bold_ref_wf(
    bold,
    split_vol_id=0,
    name='get_bold_reference_wf'
):
    """
    Get the bold reference image corresponding
    to the 4D bold image
    Parameters
    ----------
    Inputs
    ------
    Outputs
    -------
    """
    from niworkflows.engine.workflows import LiterateWorkflow as Workflow

    workflow = Workflow(name=name)

    inputnode = pe.Node(
        niu.IdentityInterface(['bold']),
        name='inputnode',
    )

    outputnode = pe.Node(
        niu.IdentityInterface(['boldref']),
        name='outputnode',
    )

    sbref = bold.replace('_bold.nii.gz','_sbref.nii.gz')
    if os.path.exists(sbref):

        workflow.connect([(inputnode, outputnode, [(('bold',_get_sbref),'boldref')])])

        return workflow

    else:

        from nipype.interfaces.fsl.utils import Split

        split_bold = pe.Node(
            Split(
                dimension='t',
                out_base_name='split_bold_'
            ),
            name='split_bold'
        )

        workflow.connect([
            (inputnode, split_bold, [('bold','in_file')]),
            (split_bold, outputnode, [(('out_files',_get_split_volume,split_vol_id),'boldref')])
        ])
        
        return workflow

In [3]:
def _index_list(X,idx):
    return X[idx]

In [4]:
workflow = Workflow(
    name = 'test_nordic',
    base_dir = '/notebooks'
)

inputnode = pe.Node(
    niu.IdentityInterface(
        [
            'anat_image',
            'mag_image',
            'phase_image',
            'out_image',
            'patch_size',
            'n_threads',
        ]
    ),
    name='inputnode'
)


fs_t1w_brain = '/data/sub-000_ses-HCoilPilotPRF_acq-MP2RAGE_run-01_desc-preproc_T1w.nii.gz'
bold_data_mag = '/data/sub-000_ses-OPCoilCBPilot01_task-AttendAwayQ1_acq-mb4_dir-RL_run-01_part-mag_bold.nii.gz'
bold_data_phase = '/data/sub-000_ses-OPCoilCBPilot01_task-AttendAwayQ1_acq-mb4_dir-RL_run-01_part-phase_bold.nii.gz'

inputnode.inputs.anat_image = fs_t1w_brain
inputnode.inputs.mag_image = bold_data_mag
inputnode.inputs.phase_image = bold_data_phase
inputnode.inputs.patch_size = [21,21,16]
inputnode.inputs.n_threads = 8

bold_ref = init_bold_ref_wf(
    bold_data_mag,
    name='bold_reference_wf'
)

resample_reference = pe.Node(
    GenerateSamplingReference(),
    name='anat_resample'
)

nordic_proc = pe.Node(
    Nordic(out_image='nordic_processed'),
    name='nordic_process'
)

nordic_gzip = pe.Node(
    Gzip(mode="compress"),
    name='nordic_gzip'
)

nordic_tsnr = pe.Node(
    TSNR(),
    name='nordic_tsnr'
)

raw_tsnr = pe.Node(
    TSNR(),
    name='raw_tsnr'
)

workflow.connect([
    (inputnode,bold_ref,[('mag_image','inputnode.bold')]),
    (bold_ref,resample_reference,[('outputnode.boldref','moving_image')]),
    (inputnode,resample_reference,[('anat_image','fixed_image')]),
    (inputnode,nordic_proc,[
        ('mag_image','mag_image'),
        ('phase_image','phase_image'),
        (('patch_size',_index_list,0),'patch_x_dim'),
        (('patch_size',_index_list,1),'patch_y_dim'),
        (('patch_size',_index_list,2),'patch_z_dim'),
        ('n_threads','n_threads'),
    ]),
    (nordic_proc,nordic_gzip,[('out_image','in_file')]),
    (nordic_gzip,nordic_tsnr,[('out_file','in_file')]),
    (inputnode,raw_tsnr,[('mag_image','in_file')]),
])

workflow.run()

230322-04:06:45,534 nipype.workflow INFO:
	 Workflow test_nordic settings: ['check', 'execution', 'logging', 'monitoring']
230322-04:06:45,539 nipype.workflow INFO:
	 Running serially.
230322-04:06:45,540 nipype.workflow INFO:
	 [Node] Setting-up "test_nordic.anat_resample" in "/notebooks/test_nordic/anat_resample".
230322-04:06:45,543 nipype.workflow INFO:
	 [Node] Executing "anat_resample" <niworkflows.interfaces.nibabel.GenerateSamplingReference>
230322-04:06:48,352 nipype.workflow INFO:
	 [Node] Finished "anat_resample", elapsed time 2.807114s.
230322-04:06:48,355 nipype.workflow INFO:
	 [Node] Setting-up "test_nordic.nordic_process" in "/notebooks/test_nordic/nordic_process".
230322-04:06:48,357 nipype.workflow INFO:
	 [Node] Executing "nordic_process" <pynordic.interfaces.nordic.Nordic>
230322-04:06:48,377 nipype.interface INFO:
	 stdout 2023-03-22T04:06:48.377649:------------------------------------------
230322-04:06:48,378 nipype.interface INFO:
	 stdout 2023-03-22T04:06:48.37